In [3]:
%%capture
%pip install -r requirements.txt

In [4]:
import os
import openai
from string import Template
import json
from neo4j import GraphDatabase
import glob
import time
from timeit import default_timer as timer
from dotenv import load_dotenv
from time import sleep
import fitz
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
import csv
import requests
import urllib.parse
from chemspipy import ChemSpider
import pubchempy as pcp
import re


## 1. Text Preprocessing Action

#### Defining Functions

In [ ]:
# Function to convert any PDF to a txt file (same folder, same name)
def convert_pdf_to_text(pdf_folder, pdf_name):
    txt_filename = pdf_name.replace(".pdf", ".txt")

    pdf_path = os.path.join(pdf_folder, pdf_name)
    txt_path = os.path.join(pdf_folder, txt_filename)

    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(text)
    
    print(f"Text extracted and saved to {txt_path}")

# Function to split one file into files with <40k characters each
def split_files(file_path):

    # Get the filename without the extension
    folder_name = os.path.splitext(file_path)[0]

    # Create a new folder with the same name as the file
    os.makedirs(folder_name, exist_ok=True)

    # Read the content of the original text file
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    
    print(text[100])

    # Define the max characters per file
    max_chars = 10000

    # Split the text into chunks of max_chars
    for i in range(0, len(text), max_chars):
        chunk = text[i:i + max_chars]
        output_file = os.path.join(folder_name, f"part_{i//max_chars + 1}.txt")
        with open(output_file, "w", encoding="utf-8") as out_f:
            out_f.write(chunk)

    print(f"Split {file_path} into {folder_name}/part_X.txt files successfully.")


# Function to find all files of one type in a folder
def find_files(folder_path, type):
    pdf_files = []
    
    # Check if the folder path exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' does not exist.")
        return pdf_files
    
    # List all files in the directory
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        
        # Check if it's a file (not a directory) and has .pdf extension
        if os.path.isfile(file_path) and file.lower().endswith(type):
            pdf_files.append(file_path)
    
    return pdf_files



# Function to create sub-folders with smaller text files <40k characters for each PDF file 
def split_pdfs(folder_path):
    pdf_files = find_files(folder_path=folder_path, type=".pdf")

    for file in pdf_files:
        convert_pdf_to_text(folder_path, file)

        file_txt = os.path.splitext(file)[0] + ".txt"
        print(file_txt)

        split_files(file_txt)

        print("Created subfolders with smaller text files for file:" + file)

In [55]:
# Same thing, but for CSV instead of PDFs

# Function to convert a CSV file to a text file (in the same folder with the same base name)
def convert_csv_to_text(csv_folder, csv_name):
    txt_filename = csv_name.replace(".csv", ".txt")
    
    csv_path = os.path.join(csv_folder, csv_name)
    txt_path = os.path.join(csv_folder, txt_filename)
    
    with open(csv_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Convert CSV content to text
        lines = []
        for row in csv_reader:
            # Join fields with tabs or another delimiter of your choice
            lines.append('\t'.join(row))
        
        text = '\n'.join(lines)
    
    with open(txt_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)
    
    print(f"CSV converted and saved to {txt_path}")


# Function to create sub-folders with smaller text files <40k characters for each CSV file 
def split_csvs(folder_path):
    pdf_files = find_files(folder_path=folder_path, type=".csv")

    for file in pdf_files:
        convert_csv_to_text(folder_path, file)

        file_txt = os.path.splitext(file)[0] + ".txt"
        print(file_txt)

        split_files(file_txt)

        print("Created subfolders with smaller text files for file:" + file)



#### Calling all functions

This will search for all PDFs in the data/regulations folder and create sub-folders for each. Within each sub-folder, the text is split up into smaller .txt files with <40k characters so it can be sent to GPT-3.5 turbo in one call.

In [48]:
folder_path = "/Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations"
split_pdfs(folder_path)

Text extracted and saved to /Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/Stockholm_Convention.txt
/Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/Stockholm_Convention.txt
g
Split /Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/Stockholm_Convention.txt into /Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/Stockholm_Convention/part_X.txt files successfully.
Created subfolders with smaller text files for file:/Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/Stockholm_Convention.pdf
Text extracted and saved to /Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/REACH.txt
/Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/REACH.txt
d
Split /Users/beatwe

In [58]:
folder_path = "/Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations"
split_csvs(folder_path)

CSV converted and saved to /Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/reach-svhc.txt
/Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/reach-svhc.txt
a
Split /Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/reach-svhc.txt into /Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/reach-svhc/part_X.txt files successfully.
Created subfolders with smaller text files for file:/Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/reach-svhc.csv


### Backup: Read CSVs Manually

In [5]:
def convert_csv_to_chemical_json(csv_file_path: str, regulation_name: str, json_file_path: str):
    """
    Reads a CSV file with 'Chemical Name' and 'CAS number' headers and outputs
    a JSON file formatted for the chemical import process. Includes entries even
    if Chemical Name or CAS number is missing, but skips if BOTH are missing.

    Args:
        csv_file_path (str): The full path to the input CSV file.
        regulation_name (str): The name of the regulation to associate with
                                 all chemicals in this CSV.
        json_file_path (str): The full path where the output JSON file will be saved.

    Raises:
        FileNotFoundError: If the input CSV file cannot be found.
        IOError: If there's an error reading the CSV or writing the JSON.
        KeyError: If the required headers ('Chemical Name', 'CAS number') are
                  missing in the CSV.
    """
    print(f"Starting conversion for CSV: {csv_file_path}")
    print(f"Using Regulation Name: {regulation_name}")
    print(f"Output JSON will be saved to: {json_file_path}")

    chemical_list = []

    try:
        # Read the CSV file using DictReader
        with open(csv_file_path, mode='r', newline='', encoding='utf-8-sig') as infile:
            reader = csv.DictReader(infile, delimiter=';')

            # Check for required headers (case-insensitive)
            headers_lower = [h.lower() for h in reader.fieldnames or []] # Handle empty file case
            if not headers_lower:
                 print("Warning: CSV file appears to be empty or has no headers.")
                 # Decide if this should be an error or just result in empty JSON
                 # For now, proceed, will result in empty JSON

            if "chemical name" not in headers_lower:
                raise KeyError("CSV file is missing the required header: 'Chemical Name'")
            if "cas number" not in headers_lower:
                 raise KeyError("CSV file is missing the required header: 'CAS number'")

            # Find the actual header names used in the file
            name_header = next((h for h in reader.fieldnames if h.lower() == "chemical name"), "Chemical Name")
            cas_header = next((h for h in reader.fieldnames if h.lower() == "cas number"), "CAS number")

            alt_name_header = next((h for h in reader.fieldnames if h.lower() == "alternative name"), None)
            if alt_name_header:
                found_alt_name_header = True
                print(f"Found 'Alternative Name' column: '{alt_name_header}'")
            else:
                print("Optional 'Alternative Name' column not found.")


            line_num = 1 # Start from 1 for header row
            processed_count = 0
            skipped_fully_empty = 0
            for row in reader:
                line_num += 1
                # Get values, stripping leading/trailing whitespace
                # Default to empty string '' if cell/column is missing for a row
                chemical_name = row.get(name_header, '').strip()
                cas_number = row.get(cas_header, '').strip()

                # Skip row only if BOTH chemical name AND CAS number are empty
                if not chemical_name and not cas_number:
                    print(f"  Skipping row {line_num}: Both Chemical Name and CAS Number are empty.")
                    skipped_fully_empty += 1
                    continue
                alternative_names_str = "" # Default to empty string
                if found_alt_name_header and alt_name_header in row:
                    # Get value, strip leading/trailing whitespace from the whole string
                    alternative_names_str = row.get(alt_name_header, '').strip()

                # Create the dictionary structure for this chemical
                # Use empty string "" for missing values in the JSON output
                chemical_entry = {
                    "chemical_name": chemical_name, # Will be "" if missing in CSV
                    "CAS": cas_number,           # Will be "" if missing in CSV
                    "regulation": regulation_name,
                    "alterantive_names": alternative_names_str
                }
                chemical_list.append(chemical_entry)
                processed_count += 1

        if skipped_fully_empty > 0:
             print(f"  Skipped {skipped_fully_empty} rows because both name and CAS were empty.")
        print(f"  Successfully processed {processed_count} chemical entries from CSV.")

        # Prepare the final JSON structure
        output_data = {"chemicals": chemical_list}

        # Ensure the output directory exists
        # Use os.path.abspath to handle relative paths correctly
        abs_json_path = os.path.abspath(json_file_path)
        os.makedirs(os.path.dirname(abs_json_path), exist_ok=True)

        # Write the data to the JSON file
        with open(abs_json_path, mode='w', encoding='utf-8') as outfile:
            json.dump(output_data, outfile, indent=4) # indent=4 for pretty printing

        print(f"Successfully created JSON file: {abs_json_path}")

    except FileNotFoundError:
        print(f"Error: Input CSV file not found at {csv_file_path}")
        raise
    except KeyError as e:
        print(f"Error: Missing required header in CSV - {e}")
        raise
    except IOError as e:
        print(f"Error reading CSV or writing JSON file: {e}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

In [30]:
convert_csv_to_chemical_json("../data/raw/CSVs/Montreal.csv", "Montreal Protocol", "../data/processed/clean/Montreal.json")

Starting conversion for CSV: ../data/raw/CSVs/Montreal.csv
Using Regulation Name: Montreal Protocol
Output JSON will be saved to: ../data/processed/clean/Montreal.json
  Successfully processed 143 chemical entries from CSV.
Successfully created JSON file: /Users/oskarribbe/Documents/Master Thesis/ChemicalGraphRAG/data/processed/clean/Montreal.json


In [62]:
convert_csv_to_chemical_json("../data/raw/CSVs/Stockholm_updated.csv", "Stockholm Convention (Updated)", "../data/processed/clean/Stockholm_updated.json")

Starting conversion for CSV: ../data/raw/CSVs/Stockholm_updated.csv
Using Regulation Name: Stockholm Convention (Updated)
Output JSON will be saved to: ../data/processed/clean/Stockholm_updated.json
Found 'Alternative Name' column: 'Alternative Name'
  Successfully processed 738 chemical entries from CSV.
Successfully created JSON file: /Users/oskarribbe/Documents/Master Thesis/ChemicalGraphRAG/data/processed/clean/Stockholm_updated.json


In [32]:
convert_csv_to_chemical_json("../data/raw/CSVs/Rotterdam.csv", "Rotterdam Convention", "../data/processed/clean/Rotterdam.json")

Starting conversion for CSV: ../data/raw/CSVs/Rotterdam.csv
Using Regulation Name: Rotterdam Convention
Output JSON will be saved to: ../data/processed/clean/Rotterdam.json
  Successfully processed 89 chemical entries from CSV.
Successfully created JSON file: /Users/oskarribbe/Documents/Master Thesis/ChemicalGraphRAG/data/processed/clean/Rotterdam.json


In [36]:
convert_csv_to_chemical_json("../data/raw/CSVs/EU_2024_590_updated_2.csv", "EU 2024 (590) (updated)", "../data/processed/clean/EU_2024_590_updated.json")

Starting conversion for CSV: ../data/raw/CSVs/EU_2024_590_updated_2.csv
Using Regulation Name: EU 2024 (590) (updated)
Output JSON will be saved to: ../data/processed/clean/EU_2024_590_updated.json
Found 'Alternative Name' column: 'Alternative Name'
  Skipping row 97: Both Chemical Name and CAS Number are empty.
  Skipped 1 rows because both name and CAS were empty.
  Successfully processed 102 chemical entries from CSV.
Successfully created JSON file: /Users/oskarribbe/Documents/Master Thesis/ChemicalGraphRAG/data/processed/clean/EU_2024_590_updated.json


In [52]:
convert_csv_to_chemical_json("../data/raw/CSVs/EU_2024_573.csv", "EU 2024 (573)", "../data/processed/clean/EU_2024_573.json")

Starting conversion for CSV: ../data/raw/CSVs/EU_2024_573.csv
Using Regulation Name: EU 2024 (573)
Output JSON will be saved to: ../data/processed/clean/EU_2024_573.json
Found 'Alternative Name' column: 'Alternative Name'
  Successfully processed 46 chemical entries from CSV.
Successfully created JSON file: /Users/oskarribbe/Documents/Master Thesis/ChemicalGraphRAG/data/processed/clean/EU_2024_573.json


In [6]:
convert_csv_to_chemical_json("../data/raw/CSVs/EU_2021_1021.csv", "EU 2021 (1021)", "../data/processed/clean/EU_2021_1021.json")

Starting conversion for CSV: ../data/raw/CSVs/EU_2021_1021.csv
Using Regulation Name: EU 2021 (1021)
Output JSON will be saved to: ../data/processed/clean/EU_2021_1021.json
Found 'Alternative Name' column: 'Alternative Name'
  Successfully processed 49 chemical entries from CSV.
Successfully created JSON file: /Users/oskarribbe/Documents/Master Thesis/ChemicalGraphRAG/data/processed/clean/EU_2021_1021.json


In [18]:
convert_csv_to_chemical_json("../data/raw/CSVs/EU_2012_649.csv", "EU 2012 (649) Annex 1", "../data/processed/clean/EU_2012_649.json")

Starting conversion for CSV: ../data/raw/CSVs/EU_2012_649.csv
Using Regulation Name: EU 2012 (649) Annex 1
Output JSON will be saved to: ../data/processed/clean/EU_2012_649.json
Found 'Alternative Name' column: 'Alternative Name'
  Skipping row 167: Both Chemical Name and CAS Number are empty.
  Skipping row 168: Both Chemical Name and CAS Number are empty.
  Skipped 2 rows because both name and CAS were empty.
  Successfully processed 213 chemical entries from CSV.
Successfully created JSON file: /Users/oskarribbe/Documents/Master Thesis/ChemicalGraphRAG/data/processed/clean/EU_2012_649.json


## 2. OpenAI Action

#### Key Validation

In [ ]:
load_dotenv() 
# Load OpenAI API key from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

client = openai.OpenAI(api_key=openai_api_key)

#### Helper Functions

In [44]:
# Function to call the OpenAI API
def process_gpt(file_prompt, system_msg):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": file_prompt},
        ],
        temperature=0,
        response_format={"type": "json_object"},
    )
    return completion.choices[0].message.content

# Function to take one file and a prompt template, and return a json_object of all the CAS numbers
def extract_cas_numbers(file, prompt_template, regulation_name):
    system_msg = "You are a helpful AI assistant who extracts CAS numbers from documents."
    try:
        with open(file, "r") as f:
            text = f.read().rstrip()
            prompt = Template(prompt_template).substitute(ctext=text, cregulation_name=regulation_name)
            result = process_gpt(prompt, system_msg=system_msg)
            return result
    except Exception as e:
        print(f"Error processing {file}: {e}")

def create_json(file, regulation_name, prompt_template):
    # extract_cas_numbers is already returning a parsed dictionary
    cas_data = extract_cas_numbers(file, prompt_template)
    
    json_output = {
        "regulation": {
            "name": regulation_name,
            "chemicals": cas_data  # Add the parsed data as "chemicals" field
        }
    }

    with open('output.json', 'w') as json_file:
        json.dump(json_output, json_file, indent=4)
    return json_output

def extract_cas_from_folder(folder, regulation_prompt, regulation_name, output_file="output.json"):
    chemicals = {'chemicals': []}
    for file in glob.glob(folder + "/*.txt"):
        print(file)
        try:
            latest_result = extract_cas_numbers(file, regulation_prompt, regulation_name)
            latest_result_dict = json.loads(latest_result)

            print(type(latest_result_dict))
            print(latest_result_dict)

            chemicals = {
                'chemicals': latest_result_dict['chemicals'] + chemicals['chemicals']
            }

            print(type(chemicals))
            print(chemicals)

        except Exception as e:
            print(e)

    with open(output_file, 'w') as f:
        json.dump(chemicals, f, indent=4)

#### Prompts

*this one works for the apple legislation, might need to be adjusted for different docs*

In [45]:
regulation_prompt = """
Extract all chemical names and their CAS numbers from the text below. A chemical name is e.g. "2,4-Diaminotoluene", and a CAS number is e.g. "95-80-7". Follow these strict rules:

### **Rules for Extraction:**
0. **NEVER STOP EARLY**. Never stop extracting before the end of the text, and never stop generating the JSON before mentioning all pairs.
1. **NEVER HALLUCINATE**. Only find real matches between **chemical names** and **CAS numbers**.
2. **Do NOT infer missing CAS numbers or chemical names.**
3. **FIND AS MANY TUPLES AS POSSIBLE**. Your goal is to maximise the number of correct (name, CAS) combinations. Find all of them!
4. If a chemical name has multiple CAS numbers, create separate entries for every valid combination.
5. If a CAS number has multiple chemical names (or names are indicated by "OR"), create separate entries for every valid combination.
7. If a chemical name has no CAS number, do NOT include it in the JSON.
6. **STRICTLY follow the JSON format.** Do NOT merge or group entries. Each unique (name, CAS) combination must be a separate object.
7. The regulation name for all entries is "$cregulation_name".
---

### **Beginning of Output Format Example (This ** 

{
    "chemicals": [
        {
            "chemical_name": "Benzene",
            "CAS": "71-43-2",
            "regulation": "$cregulation_name"
        },
        {
            "chemical_name": "Mercury",
            "CAS": "7439-97-6",
            "regulation": "$cregulation_name"
        },
        {
            "chemical_name": "Lead chromate",
            "CAS": "7758-97-6",
            "regulation": "$cregulation_name"
        }
    ]
}

### **End of Output Format Example:**

In the following, you also find some bad examples, which you should avoid. If the output looks like the following, don't mention it in the JSON:

### ** Start of Bad EXAMPLES:** 

{
    "chemical_name": "PFHxA, its salts and related substances",
    "CAS": "Not Available",
    "regulation": "Apple Regulated Substances Specification 069-0135-M"
},
{
    "chemical_name": "Endocrine Disrupting Chemicals (EDCs)",
    "CAS": null,
    "regulation": "Apple Regulated Substances Specification 069-0135-M"
},
{
    "chemical_name": "Wearable products",
    "CAS": "",
    "regulation": "Apple Regulated Substances Specification 069-0135-M"
},
{
    "chemical_name": "Textile",
    "CAS": "N/A",
    "regulation": "Apple Regulated Substances Specification 069-0135-M"
},

Text to process:
$ctext
"""

In [61]:
regulation_prompt_csv = """
Extract all chemical names and their CAS numbers from the text below. A chemical name is e.g. "2,4-Diaminotoluene", and a CAS number is e.g. "95-80-7". Follow these strict rules:

### **Rules for Extraction:**
0. **NEVER STOP EARLY**. Never stop extracting before the end of the text, and never stop generating the JSON before mentioning all pairs.
1. **NEVER HALLUCINATE**. Only find real matches between **chemical names** and **CAS numbers**.
2. **Do NOT infer missing CAS numbers or chemical names.**
3. **FIND AS MANY TUPLES AS POSSIBLE**. Your goal is to maximise the number of correct (name, CAS) combinations. Find all of them!
4. If a chemical name has multiple CAS numbers, create separate entries for every valid combination.
5. If a CAS number has multiple chemical names (or names are indicated by "OR"), create separate entries for every valid combination.
6. If a chemical name has no CAS number, do NOT include it in the JSON.
7. Most pairs are in one row, seperated by a semicolone with the scheme Chemical Name; CAS. Look out for those!
8. **STRICTLY follow the JSON format.** Do NOT merge or group entries. Each unique (name, CAS) combination must be a separate object.
9. The regulation name for all entries is "$cregulation_name".---

### **Beginning of Output Format Example (This ** 

{
    "chemicals": [
        {
            "chemical_name": "Benzene",
            "CAS": "71-43-2",
            "regulation": "$cregulation_name"
        },
        {
            "chemical_name": "Mercury",
            "CAS": "7439-97-6",
            "regulation": "$cregulation_name"
        },
        {
            "chemical_name": "Lead chromate",
            "CAS": "7758-97-6",
            "regulation": "$cregulation_name"
        }
    ]
}

### **End of Output Format Example:**

Text to process:
$ctext
"""

#### Pipeline for one doc

Just specify the file and the regulation name and running this cell will produce an output.json

In [52]:
file = "/Users/beatweichsler/Documents/temp/chemicalgraph_v1/data/regulations/Apple_069-0135-M/part_3.txt"
regulation_name = "Apple Regulated Substances Specification 069-0135-M"
#file = "/Users/beatweichsler/Documents/temp/chemicalgraph_v1/data/apple_regulation/Apple_Regulated_Substances_Specification.txt"
result = extract_cas_numbers(file, regulation_prompt, regulation_name)
print(result)

result_json = json.loads(result)
print(type(result))
print(type(result_json))

{
    "chemicals": [
        {
            "chemical_name": "Lead",
            "CAS": "7439-92-1",
            "regulation": "Apple Regulated Substances Specification 069-0135-M"
        },
        {
            "chemical_name": "Lead compounds",
            "CAS": "7439-92-1",
            "regulation": "Apple Regulated Substances Specification 069-0135-M"
        },
        {
            "chemical_name": "Mercury",
            "CAS": "7439-97-6",
            "regulation": "Apple Regulated Substances Specification 069-0135-M"
        },
        {
            "chemical_name": "Mercury compounds",
            "CAS": "7439-97-6",
            "regulation": "Apple Regulated Substances Specification 069-0135-M"
        },
        {
            "chemical_name": "Methyl-phenol compounds",
            "CAS": "95-48-7",
            "regulation": "Apple Regulated Substances Specification 069-0135-M"
        },
        {
            "chemical_name": "Methyl-phenol compounds",
            "CAS": "

#### Pipeline for the entire data/regulations folder

Pipeline scans all .txt files in the folder and concatenates all chemicals 

In [62]:
folder = "/Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/reach-svhc"
regulation_name = "REACH SVHC Candidate List" 

chemicals = {'chemicals': []}
output_file = 'output_reach_svhc.json'

extract_cas_from_folder(folder, regulation_prompt_csv, regulation_name, output_file)


/Users/beatweichsler/Documents/1_uni/Master/MasterThesis/github/ChemicalGraphRAG/data/regulations/reach-svhc/part_2.txt
<class 'dict'>
{'chemicals': [{'chemical_name': '1,3-propanesultone', 'CAS': '1120-71-4', 'regulation': 'REACH SVHC Candidate List'}, {'chemical_name': 'N,N,N-triethylethanaminium tridecafluorohexane-1-sulfonate', 'CAS': '108427-55-0', 'regulation': 'REACH SVHC Candidate List'}, {'chemical_name': 'N,N,N-tributylbutan-1-aminium tridecafluorohexane-1-sulfonate', 'CAS': '108427-54-9', 'regulation': 'REACH SVHC Candidate List'}, {'chemical_name': '1,2-benzenedicarboxylic acid di-C6-10-alkyl esters', 'CAS': '68515-51-5', 'regulation': 'REACH SVHC Candidate List'}, {'chemical_name': '1,2-benzenedicarboxylic acid mixed decyl and hexyl and octyl diesters', 'CAS': '68648-93-1', 'regulation': 'REACH SVHC Candidate List'}, {'chemical_name': '2-ethylhexyl 10-ethyl-4,4-dioctyl-7-oxo-8-oxa-3,5-dithia-4-stannatetradecanoate (DOTE)', 'CAS': '15571-58-1', 'regulation': 'REACH SVHC Can

In [51]:
# Cleaning up the JSON (removing all n/a CAS numbers)

# Load the JSON data from file
with open('output_stockholm.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Filter out chemicals with various forms of "no CAS number"
filtered_chemicals = [
    chem for chem in data['chemicals'] 
    if chem.get('CAS') not in ["Not Available", "N/A", "No CAS number available", ""] 
    and chem.get('CAS') is not None
]

# Update the data structure
data['chemicals'] = filtered_chemicals

# Save the updated data back to file
with open('output_stockholm_filtered.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

print("Filtered data saved to output_reach_filtered.json")

Filtered data saved to output_reach_filtered.json


## 3. NEO4J Action

Class for interacting with the ChemicalDatabase
-> Currently only loads data

In [7]:
import re

def resolve_chemical_name(chemical_name: str) -> str | None:

        resolved_cas = None
        # ChemSpider might have different rate limits, adjust delay if needed
        api_delay = 0.1

        try:
            print(f"Querying PubChem for: '{chemical_name}'")
            
            results = pcp.get_synonyms(chemical_name, 'name')

            # --- Process ONLY the First Result ---
            if results: # Check if the results list is not empty
                for syn in results[0].get('Synonym', []):
                    match = re.match('(\d{2,7}-\d\d-\d)', syn)
                    if match:
                        print(f"PubChem Top Result - Name: {chemical_name}, CAS (attempted): '{match}',")
                        resolved_cas=match.group(1)
                        break
                

            else:
                print(f"ChemSpider returned no results for '{chemical_name}'.")

        except Exception as e:
            print(f"Error during PubChem query for '{chemical_name}': {e}")
            print(f"Exception type: {type(e)}") # Log exception type for debugging


        # Add a small delay after the operation
        time.sleep(api_delay)

        if not resolved_cas:
            print(f"Could not resolve '{chemical_name}' to a validated CAS number via ChemSpider Top Result.")

        return resolved_cas # Return CAS string or None

In [50]:
resolve_chemical_name("HFE-134")

Querying PubChem for: 'HFE-134'
ChemSpider returned no results for 'HFE-134'.
Could not resolve 'HFE-134' to a validated CAS number via ChemSpider Top Result.


In [8]:
# --- NEW METHOD ---
def find_names_for_cas(cas_number: str, limit: int = 3) -> list[str]:
    """
    Attempts to find common names for a given CAS number using PubChem
    via the pubchempy package. Returns up to 'limit' names.

    Args:
        cas_number (str): The CAS number to look up.
        limit (int): The maximum number of names to return.

    Returns:
        list[str]: A list of chemical names (synonyms), excluding the CAS itself.
    """
    if not cas_number: return []

    names_found = []
    api_delay = 0.2
    # Ensure input CAS looks valid before querying
    cas_pattern = re.compile(r'^\d{2,7}-\d{2}-\d$')
    if not cas_pattern.match(cas_number):
        print(f"Skipping name lookup for invalid CAS format: '{cas_number}'")
        return []

    try:
        # print(f"Querying PubChem via pubchempy for CAS: '{cas_number}'") # Optional verbose log
        # Use 'cas' namespace for lookup
        results = pcp.get_compounds(cas_number, 'name')

        if results:
            # CAS lookups should be specific, use the first result
            compound = results[0]
            # print(f"PubChemPy Found Compound - CID: {getattr(compound,'cid','N/A')}") # Optional log

            if hasattr(compound, 'synonyms') and compound.synonyms:
                count = 0
                for synonym in compound.synonyms:
                    # Ensure synonym is a non-empty string and NOT the CAS number itself
                    if isinstance(synonym, str) and synonym.strip() and synonym != cas_number and not cas_pattern.match(synonym):
                        names_found.append(synonym.strip().title())
                        count += 1
                        if count >= limit:
                            break # Stop once we reach the desired limit
                # print(f"  Found {len(names_found)} potential names in synonyms.") # Optional log
            # else: print(f"  No synonyms found for compound CID: {getattr(compound,'cid','N/A')}") # Optional log
        # else: print(f"PubChemPy returned no compound results for CAS '{cas_number}'.") # Optional log

    except pcp.PubChemHTTPError as e:
            print(f"PubChem HTTP Error for CAS '{cas_number}': {e}")
    except TimeoutError:
            print(f"Timeout during PubChem query for CAS '{cas_number}'.")
    except Exception as e:
        print(f"Error querying PubChem (CAS) via pubchempy for '{cas_number}': {type(e).__name__} - {e}")

    time.sleep(api_delay)
    return names_found

In [21]:
find_names_for_cas("75-72-9")

['Chlorotrifluoromethane',
 'Trifluoromethyl Chloride',
 'Trifluorochloromethane']

In [9]:
class ChemicalDatabase:
    def __init__(self, URI, AUTH):
        self._driver = GraphDatabase.driver(URI, auth=AUTH)

    def close(self):
        self._driver.close()

    def import_json(self, chemicals_data_json):
        """
        Inserts chemical data from a JSON-like structure into Neo4j.

        Args:
            chemicals_data (dict): A dictionary containing a "chemicals" key with a list of chemical dictionaries.
        """
        if not isinstance(chemicals_data_json, dict) or "chemicals" not in chemicals_data_json or not isinstance(chemicals_data_json["chemicals"], list):
            raise ValueError("Invalid chemicals data format. Expected a dictionary with a 'chemicals' list.")

        with self._driver.session() as session:
            for chemical in chemicals_data_json["chemicals"]:
                self._insert_chemical(session, chemical, resolve_chemical_name)

    @staticmethod
    def _insert_chemical(session, chemical, resolver_func):
        """
        Inserts a single chemical into Neo4j, handling cases with missing CAS or chemical name.

        Args:s
            session (neo4j.Session): The Neo4j session.
            chemical (dict): A dictionary representing a single chemical.
        """
        chemical_name = chemical.get("chemical_name")
        cas_raw = chemical.get("CAS") 
        regulation = chemical.get("regulation")

        invalid_cas_values = {"N/A", "NONE", "NOT APPLICABLE", "na", "NA", ""}

        cas=None
        if isinstance(cas_raw, str):
            normalized_cas = cas_raw.strip().upper()
            if normalized_cas not in invalid_cas_values:
                cas = cas_raw.strip()

        if chemical_name and cas==None:
            print(f"Attempting to resolve name: '{chemical_name}'")
            resolved_cas = resolver_func(chemical_name=chemical_name)

            if resolved_cas:
                cas = resolved_cas # Use the resolved CAS for subsequent logic
                print(f"Successfully used resolved CAS: {cas} for '{chemical_name}'")
            else:
                # Resolution failed or returned invalid CAS
                print("No CAS found")


        if not regulation:
            print(f"Skipping chemical due to missing regulation: {chemical}")
            return

        if chemical_name and cas:
            query = """
            MERGE (c:Chemical {cas: $cas})
            MERGE (cn:ChemicalName {name: $name})
            MERGE (r:Regulation {name: $regulation})
            MERGE (cn)-[:IS_NAME_OF]->(c)
            MERGE (cn)-[:IS_REGULATED]->(r)
            MERGE (c)-[:IS_REGULATED]->(r)
            """
            session.run(query, name=chemical_name, cas=cas, regulation=regulation)

        elif chemical_name:

            query = """
            MERGE (cn:ChemicalName {name: $name})
            MERGE (r:Regulation {name: $regulation})
            MERGE (cn)-[:IS_REGULATED]->(r)
            """
            session.run(query, name=chemical_name, regulation=regulation)

        elif cas:
            query = """
            MERGE (c:Chemical {cas: $cas})
            MERGE (r:Regulation {name: $regulation})
            MERGE (c)-[:IS_REGULATED]->(r)
            """
            session.run(query, cas=cas, regulation=regulation)

            found_names = find_names_for_cas(cas) # Limit defaults to 3

            # 3. If names found, merge them and link to Chemical
            if found_names:
                print(f"  Found names for CAS {cas}: {found_names}. Merging...")
                # Loop and run merge for each name
                # Using UNWIND is more efficient for multiple names
                query_names = """
                MATCH (c:Chemical {cas: $cas}) // Match the chemical created above
                UNWIND $names_list AS name     // Unpack the list of names
                MERGE (cn:ChemicalName {name: name}) // Create/find ChemicalName node
                MERGE (cn)-[:IS_NAME_OF]->(c)       // Create/find link to Chemical
                SET cn.needs_curation = false      // Mark these found names as resolved
                """
                session.run(query_names, cas=cas, names_list=found_names)
                # Alternative (less efficient for many names):
                # for name in found_names:
                #     query_single_name = """
                #     MATCH (c:Chemical {cas: $cas})
                #     MERGE (cn:ChemicalName {name: $name})
                #     MERGE (cn)-[:IS_NAME_OF]->(c)
                #     SET cn.needs_curation = false
                #     """
                #     tx.run(query_single_name, cas=cas, name=name)
            else:
                print(f"  No additional names found via PubChem for CAS {cas}.")

        else:
            print(f"Skipping chemical due to missing chemical name and CAS: {chemical}")

## Load Data into database

In [10]:
load_status = load_dotenv("../auraconnection.txt")

if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

In [11]:
# Verify Connectivity to NEO4J
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [45]:
with open('../data/processed/clean/EU_2021_1021.json', 'r') as f:
    chemicals_data_json = json.load(f)

db = ChemicalDatabase(URI, AUTH)
try:
    db.import_json(chemicals_data_json)
    print("Chemicals inserted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    db.close()

  Found names for CAS 2795-39-3: ['Potassium Perfluorooctanesulfonate', 'Perfluorooctanesulfonic Acid Potassium Salt', 'Potassium Heptadecafluoro-1-Octanesulfonate']. Merging...
  Found names for CAS 29457-72-5: ['Lithium Perfluorooctane Sulfonate', 'Lithium Perfluorooctanesulfonate', 'Lithium Pfos']. Merging...
  Found names for CAS 29081-56-9: ['Ammonium 1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,8-Heptadecafluorooctane-1-Sulfonate', 'Ammonium Heptadecafluoro-1-Octanesulfonate', 'Ammonium Pfos']. Merging...
  Found names for CAS 70225-14-8: ['Bis(2-Hydroxyethyl)Ammonium Perfluorooctanesulfonate', 'Y5Gtm594K0', 'Diethanolamine Perfluoroctanesulfonate']. Merging...
  Found names for CAS 56773-42-3: ['Tetraethylammonium Perfluorooctanesulfonate', 'Tetraethylammonium Heptadecafluorooctanesulphonate', 'Tetraethylammonium 1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,8-Heptadecafluorooctane-1-Sulfonate']. Merging...
  Found names for CAS 251099-16-8: ['1-Decanaminium, N-Decyl-N,N-Dimethyl-, 1,1,2,2,3,3,4,4,5,5,6,6

## RAG Test

- uses the Langchain Neo4J wrapper to get a graph object
- Use the Google API (also works with ChatOpenAI) to generate Cypher Queries

In [70]:
load_status = load_dotenv()
if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

API_KEY = os.getenv("API_KEY")

graph = Neo4jGraph(
    url=URI,
    username=AUTH[0],
    password=AUTH[1]
)

chain = GraphCypherQAChain.from_llm(
    ChatGoogleGenerativeAI(temperature=0, model="gemini-2.0-flash",google_api_key=API_KEY, allow_dangerous_requests=True), graph=graph, verbose=True, allow_dangerous_requests=True
)

chain.run("How is CAS 7440-43-9 regulated?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Chemical {cas: "7440-43-9"})-[:IS_REGULATED]->(r:Regulation)
RETURN r.name

Full Context:
[{'r.name': 'REACH SVHC Candidate List'}, {'r.name': 'HM Group Chemical Restrictions'}, {'r.name': 'Apple Regulated Substances Specification 069-0135-M'}]

> Finished chain.


'CAS 7440-43-9 is regulated by REACH SVHC Candidate List, HM Group Chemical Restrictions, and Apple Regulated Substances Specification 069-0135-M.'

## Legacy Neo4J functions

In [4]:
class Neo4jChemicalGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_regulation(self, name):
        """Create a regulation node."""
        query = """
        MERGE (r:Regulation {name: $name})
        RETURN r
        """
        with self.driver.session() as session:
            session.run(query, name=name)

    def create_chemical(self, name, cas=None, ec=None):
        """Create a Chemical node if CAS is provided, otherwise just create a ChemicalName node."""
        if cas:
            query = """
            MERGE (c:Chemical {CAS: $cas})
            ON CREATE SET c.EC = $ec
            MERGE (cn:ChemicalName {name: $name})
            MERGE (cn)-[:BELONGS_TO]->(c)
            RETURN c, cn
            """
            with self.driver.session() as session:
                session.run(query, name=name, cas=cas, ec=ec)
        else:
            # Only create ChemicalName if CAS is missing
            query = """
            MERGE (cn:ChemicalName {name: $name})
            RETURN cn
            """
            with self.driver.session() as session:
                session.run(query, name=name)

    def link_regulation_to_chemical(self, reg_name, cas):
        """Link a regulation to an existing chemical."""
        query = """
        MATCH (c:Chemical {CAS: $cas}), (r:Regulation {name: $reg_name})
        MERGE (c)-[:REGULATED_BY]->(r)
        """
        with self.driver.session() as session:
            session.run(query, cas=cas, reg_name=reg_name)

    def link_regulation_to_chemical_name(self, reg_name, chem_name):
        """Link a regulation to an existing chemical name."""
        query = """
        MATCH (cn:ChemicalName {name: $chem_name}), (r:Regulation {name: $reg_name})
        MERGE (cn)-[:REGULATED_BY]->(r)
        """
        with self.driver.session() as session:
            session.run(query, chem_name=chem_name, reg_name=reg_name)

    def import_json(self, json_data):
        """Import JSON into Neo4j while handling cases with missing CAS/EC numbers."""
        regulation_name = json_data["regulation"]["name"]
        self.create_regulation(regulation_name)

        for chemical in json_data["regulation"]["chemicals"]:
            chem_name = chemical["name"]
            cas = chemical.get("cas")  # CAS might be missing
            ec = chemical.get("ec")   # EC might be missing

            # Create chemical and chemical name if they do not exist
            self.create_chemical(chem_name, cas, ec)
            
            # Link regulation to existing chemical (if CAS is provided) or chemical name (in case only name is provided)
            if cas:
                self.link_regulation_to_chemical(regulation_name, cas)
            self.link_regulation_to_chemical_name(regulation_name, chem_name)

In [ ]:
# NEO4J AURA configuration
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# # Verify Connectivity to NEO4J
# with GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD)) as driver:
#     driver.verify_connectivity()

# # Set Up Database
# db = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

db = Neo4jChemicalGraph(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)

#### Helper Functions

In [ ]:
with open('output.json', 'r') as json_file:
    json_data = json.load(json_file)

db.import_json(json_data)

In [ ]:
# chemicals_data_json = {
#         "chemicals": [
#             {
#                 "chemical_name": "Lead",
#                 "CAS": "7439-92-1",
#                 "regulation": "Apple Regulated Substances Specification 069-0135-M"
#             },
#             {
#                 "chemical_name": "Lead compounds",
#                 "CAS": "7439-92-1",
#                 "regulation": "Apple Regulated Substances Specification 069-0135-M"
#             },
#             {
#                 "chemical_name": "Mercury",
#                 "CAS": "7439-97-6",
#                 "regulation": "Apple Regulated Substances Specification 069-0135-M"
#             },
#             {
#                 "chemical_name": "Mercury compounds",
#                 "CAS": "7439-97-6",
#                 "regulation": "Apple Regulated Substances Specification 069-0135-M"
#             },
#             {
#                 "chemical_name": "Arsenic",
#                 "regulation": "Apple Regulated Substances Specification 069-0135-M"
#             },
#             {
#                 "CAS": "123-45-6",
#                 "regulation": "Another regulation"
#             },
#             {
#                 "CAS": "7439-97-6",
#                 "regulation": "Another regulation"
#             },
#         ]
#     }

In [ ]:
def resolve_chemical_name(chemical_name: str) -> str | None:
        """
        Attempts to resolve a chemical name to a CAS number using ONLY the
        first result from the CAS Common Chemistry API. PubChem fallback is removed.

        Args:
            chemical_name (str): The chemical name to resolve.

        Returns:
            str | None: The CAS number ('rn') string from the first result
                      if found and minimally validated, otherwise None.
        """
        if not chemical_name:
            return None

        headers = {'Accept': 'application/json'}
        resolved_cas = None
        # Keep delay for politeness towards the API, even if only one call per name
        api_delay = 0.2

        # --- Query CAS Common Chemistry API ---
        try:
            search_query = urllib.parse.quote(chemical_name)
            cc_url = f"https://commonchemistry.cas.org/api/search?q={search_query}"
            # Consider adding parameters like size=1 if the API supports limiting results
            # cc_url = f"https://commonchemistry.cas.org/api/search?q={search_query}&size=1"
            print(f"Querying CAS Common Chemistry: {cc_url}")

            response = requests.get(cc_url, headers=headers, timeout=15)
            response.raise_for_status() # Check for HTTP errors (4xx, 5xx)
            data = response.json()

            # --- Process ONLY the First Result ---
            # Check if count > 0 and the results list exists and is not empty
            if data.get('count', 0) > 0 and 'results' in data and data['results']:
                first_result = data['results'][0] # Directly access the first result

                result_name = first_result.get('name', '').strip()
                result_cas = first_result.get('rn') # Get CAS ('rn') from the first result

                print(f"CAS CC Top Result - Name: '{result_name}', CAS (rn): '{result_cas}'")

            else:
                 print(f"CAS CC returned no results for '{chemical_name}'.")

        except requests.exceptions.Timeout:
             print(f"API Timeout querying CAS Common Chemistry for '{chemical_name}'.")
        except requests.exceptions.RequestException as e:
            print(f"API Error querying CAS Common Chemistry for '{chemical_name}': {e}")
        except Exception as e: # Catch other errors like JSONDecodeError
             print(f"Error processing CAS Common Chemistry response for '{chemical_name}': {e}")

        # Add a small delay after the API call
        time.sleep(api_delay)

        # --- PubChem Fallback Removed ---

        # Final status log
        if not resolved_cas:
            print(f"Could not resolve '{chemical_name}' to a validated CAS number via CAS CC Top Result.")

        return resolved_cas